In [ ]:
# final
from gurobipy import *
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from typing import List
import pandas as pd
from gurobipy import GRB
import numpy as np

In [ ]:
def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw




def distinctCustomer(path,nodeCount):
    customer = 0
    visited = [] # stores index of visited customer
    for i in range(1,len(path)-1):
        node = path[i]
        for j in range(0,len(nodeCount)):
            if(nodeCount[j][0][0] <= node  and node <= nodeCount[j][-1][-1]):
                if(j not in visited):
                    visited.append(j)
                    customer+=1
    return customer,visited


def maxCustomerpath(paths,nodeCount):
    maxPaths = 0
    maxCustomer = 0
    for i in range(0,len(paths)):
        path = paths[i]
        customer,visited = distinctCustomer(path,nodeCount)
        if(maxCustomer < customer):
            maxCustomer = customer
            maxPaths = visited # customers visited
            pathdirection = path
            
    return maxCustomer,maxPaths,pathdirection

def mycallback(model, where):

    if where == GRB.Callback.BARRIER:
        # Barrier callback
        itcnt = model.cbGet(GRB.Callback.BARRIER_ITRCNT)
        primobj = model.cbGet(GRB.Callback.BARRIER_PRIMOBJ)
        dualobj = model.cbGet(GRB.Callback.BARRIER_DUALOBJ)
        priminf = model.cbGet(GRB.Callback.BARRIER_PRIMINF)
        dualinf = model.cbGet(GRB.Callback.BARRIER_DUALINF)
        cmpl = model.cbGet(GRB.Callback.BARRIER_COMPL)
        if abs(primobj - dualobj) <= 5:
            print("Custom gap reached")
            model.terminate()
        print("%d %g %g %g %g %g" % (itcnt, primobj, dualobj, priminf, dualinf, cmpl))
        
def getCustomer(node,nodeCount):
    for i in range(len(nodeCount)):
            smallest_node = nodeCount[i][0][0]
            largest_node = nodeCount[i][-1][-1]
            if(nodeCount[i][0][0] <= node  and node <= nodeCount[i][-1][-1]):
                return i
         
    
def length(u,v,nodeCount,path,idx,ts):
    cust_u = getCustomer(u,nodeCount)
    cust_v = getCustomer(v,nodeCount)
    if(cust_u == cust_v):
        return 0
    # x will start from v and go back to see if cust(x) = cust_u
#     route = []
#     route.append(v)
    curr = idx
    while(True):
        if(curr == 0):
            break
        prev = path[curr]
#         route.append(ts[prev])
        if(getCustomer(ts[prev],nodeCount) == cust_u):
            return 0
        curr = prev
    return 1
        
        
def dfs(v,visited,stack,graph):
        visited[v] = True
        for i in graph[v]:
            if visited[i] == False:
                dfs(i,visited,stack,graph)
        stack.insert(0,v)
def topologicalSort(visited,graph):
        stack =[] 
        dfs(0,visited,stack,graph)
        print ("topologically sorted",stack)
        return stack

def onesPath(graph: List[List[int]],edges1:List[List[int]]) -> List[List[int]]:
    stack = [[0]]
    s=[[]]
    paths = []
    ed=[]
    while stack:
        path = stack.pop()
        e=s.pop()
        node = path[-1]
        if node == len(graph) - 1:
            paths.append(path)
            ed.append(e)
            continue
            
        for i in range(len(graph[node])):
            if(ans[edges1[node][i]]==1):
                stack.append((path + [graph[node][i]]))
                s.append((e+ [edges1[node][i]]))       
            
    return  paths
def searchKey(customer,cust_dict):
    for key in cust_dict.keys():        
        if customer == cust_dict[key]:
            return k

def distance_cal(route,df):
    dis=df['XCOORD.'][route[1]]
    for i in range(1,len(route)-1):
        dis+=math.sqrt((df['XCOORD.'][route[i+1]]-df['XCOORD.'][route[i]])**2 +(df['YCOORD.'][route[i+1]]-df['YCOORD.'][route[i]])**2)
    return dis

In [ ]:
customers = 100
file = "C103.csv"
df = pd.read_csv(file) 
start = 0
speed = 1 # km/min
solomon_capacity = 200


df['XCOORD.'] = df['XCOORD.'] - df['XCOORD.'][0]
df['YCOORD.'] = df['YCOORD.'] - df['YCOORD.'][0]
# df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
# df.reset_index(inplace = True)

serviceTime = df['SERVICE TIME'][1]
#remove unreachable if(starting time + travelling > end )


print(len(df))
# df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
timeWindows = [] 
for i in range(0,customers):
    st = df['READY TIME'][i+1]
    if(st%10 != 0):
        st=st + (10 - df['READY TIME'][i+1]%10)
    et = df['DUE DATE'][i+1]
    if(et%10 != 0):
        et = et +(10 - df['DUE DATE'][i+1]%10)
    tw = [st,et]
    timeWindows.append(tw)

discreteTimeWindows = [] # [ [] , [] , [] ]
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i,10))


def discreteCapacities(C,discrete):
    cap = []
    currC = discrete
    while (currC <= C):
            cap.append(currC)
            currC = currC+discrete
    return cap

capacities = [] #3x3 to store capacities , capacities[i][j][k] -> capacity of ith customer @ discreteTimeWindows[i][j]
for i in range(0,len(discreteTimeWindows)):
    customer_i = []
    dc = discreteCapacities(solomon_capacity,10)
    for j in range(0,len(dc)):
        customer_i.append([dc[j]] * len(discreteTimeWindows[i]))
    capacities.append(customer_i)

time_from_source = []
for i in range(0,customers):
    time_from_source.append(round(math.sqrt((df['XCOORD.'][i+1])**2  + (df['YCOORD.'][i+1])**2 ),2))


demand = []
for i in range(0,customers):
    if(df['DEMAND'][i+1]%10 != 0):#considering hard-demands
        demand.append(df['DEMAND'][i+1] + (10-df['DEMAND'][i+1]))
    else:
        demand.append(df['DEMAND'][i+1])
    

timeMatrix = [] # time from one customer node to another
for i in range(0,customers):
    timeMatrix.append([0]*customers)
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
            euclidean_distance= round( math.sqrt((df['XCOORD.'][i+1]-df['XCOORD.'][j+1])**2  + (df['YCOORD.'][i+1]-df['YCOORD.'][j+1])**2 ),2)
            timeMatrix[i][j] = round(euclidean_distance,2)
            timeMatrix[j][i] = round(euclidean_distance,2)
# print(discreteTimeWindows)
# capacities

In [ ]:
total_runtime = 0
cust_dict = {}
for i in range(customers):
    cust_dict[i] = i

final_path_list = []
Iterations = 0
distance=    
VEHICLES = 0
customers_covered = 0

while(len(timeWindows)!=0):
    Iterations+=1
    m1 = Model('VRPTW')
    
    count = 0
    nodeCount = []  # assigns a number to node using count and stores its value

    
    for i in range(0, len(capacities)):
        customer_i = []
        for j in range(0, len(capacities[i])):
            customer_i_j = []
            for k in range(0, len(capacities[i][j])):
                customer_i_j.append(count+1)
                count += 1
            customer_i.append(customer_i_j)
        nodeCount.append(customer_i)

#     print("Number of customers",len(nodeCount))
#     print("Nodes ",nodeCount)
#     print("time from source",time_from_source)

  #  break

    edgeFlows = {} 
    edgeNumber = 0
    x = [] 
    edges = {}  # x[edge #] = (u,v)
    edgesList = []  # stores edge number going out of ith node
    edgesList.append([])  # for s
    
    
    
    # GRAPH CONSTRUCTION
    
    # out of s to starting of each customer
    e = []
    for i in range(0,len(nodeCount)):
        reaching_time = time_from_source[i]
        demand_i = demand[i] 
        for j in range(0, len(discreteTimeWindows[i])):
                        if(reaching_time <= discreteTimeWindows[i][j]):
                            reaching_time_index = j
                            break
        capacity_index = dc.index(0+demand_i)    
    
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
        edges['x[%i]' % edgeNumber] = (0,nodeCount[i][capacity_index][reaching_time_index])
        edgeFlows[edgeNumber] = (0,nodeCount[i][capacity_index][reaching_time_index])
        e.append(edgeNumber)
        edgeNumber += 1
                         
    edgesList[0] = e
   

    # edges between consecutive nodes
    for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])):
            for k in range(0,len(nodeCount[i][j])-1):
                x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                edges['x[%i]' % edgeNumber] = (nodeCount[i][j][k], nodeCount[i][j][k+1])
                edgeFlows[edgeNumber] = (nodeCount[i][j][k], nodeCount[i][j][k+1])
                edgesList.append([edgeNumber])
                edgeNumber += 1
            edgesList.append([])


# edge construction (main)
    for i in range(len(nodeCount)):
        for j in range(len(nodeCount[i])):

            tu = nodeCount[i][j][-1]
            start_utime = discreteTimeWindows[i][0]
            end_utime = discreteTimeWindows[i][-1]
            demand_u = demand[i] 
            capacity_u = dc[j] # cap @ node u


            for k in range(len(nodeCount)):
                for l in range(len(nodeCount[k])):

                    sv = nodeCount[k][l][0] # first node of each capacity
                    start_vtime = discreteTimeWindows[k][0]
                    end_vtime = discreteTimeWindows[k][-1]
                    demand_v = demand[k] 
                    capacity_v = dc[l] # cap @ node l


                    if(i != k and start_utime + timeMatrix[i][k] + serviceTime > end_vtime):
                        print("",end='')
                      # not reachable

                    # reaches before sv starting from the last node of u
                    elif(i != k and start_vtime >= end_utime + timeMatrix[i][k]+ serviceTime and capacity_u+demand_v==capacity_v):
                        e = []
                        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                        edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
                        edgeFlows[edgeNumber] = (tu, sv)
                        e.append(edgeNumber)
                        edgesList[tu].extend(e)
                        edgeNumber += 1


                    elif (i != k and start_vtime < end_utime + timeMatrix[i][k]+serviceTime and capacity_u+demand_v == capacity_v):

                        firstReachableNode_index = 0  # first such that node >= sv
                        for m in range(0, len(discreteTimeWindows[i])):
                            if(timeMatrix[i][k] + discreteTimeWindows[i][m]+serviceTime >= start_vtime):
                                firstReachableNode_index = m
                                break
                        if(firstReachableNode_index > 0):
                                firstReachableNode_index=firstReachableNode_index-1
                        # now starting from firstReachableNode make edges until feasible
                        for n in range(firstReachableNode_index, len(discreteTimeWindows[i])):
                            e = []
                            reachingTime = discreteTimeWindows[i][n]+timeMatrix[i][k]+serviceTime
                            for r in range(0, len(discreteTimeWindows[k])):
                                if(discreteTimeWindows[k][r] >= reachingTime):
                                    x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                                    edges['x[%i]' % edgeNumber] =(nodeCount[i][j][n], nodeCount[k][l][r])  # u -> v reachable
                                    edgeFlows[edgeNumber] = (nodeCount[i][j][n], nodeCount[k][l][r])
                                    e.append(edgeNumber)
                                    edgesList[nodeCount[i][j][n]].extend(e)
                                    edgeNumber += 1
                                    break



#     print('edge list b4 s t', edgesList)
#     print()

    # all ti to t
    e = []
    for i in range(0, len(nodeCount)):
        for j in range(0,len(nodeCount[i])):
            ti = nodeCount[i][j][-1]
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
            edges['x[%i]' % edgeNumber] = (ti , -1)
            edgeFlows[edgeNumber] = (ti , -1)
            # add edge number in edge list
            edgesList[ti].append(edgeNumber)
            edgeNumber += 1
    edgesList.append([])


#     print(edgesList)
    # GRAPH Construction OVER
    # if I add it to  0 edge i'll get flow for 1 at the same edge  , add to 1 customer 2 customer


    offset = edgeNumber


#     flows = [0]*customers
#     for i in range(customers):
#         flows[i] = edgeFlows.copy()
#         keys = edgeFlows.keys()
#         for j in keys:  # stores key values of edgeflows i.e = 0,1,2....
#             x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
#             # remove key and replace it by edge
#             t = flows[i].pop(j)
#             flows[i][edgeNumber] = t
#             edgeNumber += 1

    noc = 0
    # contraint for outgoing flow for a customer
    for i in range(0, len(nodeCount)):  # choose one list from nodeCount
        smallest_node = nodeCount[i][0][0]
        largest_node = nodeCount[i][-1][-1]
        coeff = []
        for j in range(0, len(nodeCount[i])):
            for k in range(0,len(nodeCount[i][j])):
                node = nodeCount[i][j][k]
                for l in range(0,len(edgesList[node])):
                    e = edgesList[node][l]
                    u = edges['x[%i]' % e][0]
                    v = edges['x[%i]' % e][1]
                    if(v<smallest_node or v>largest_node):
                            coeff.append(x[e])
        m1.addConstr(( sum(coeff) == 1  ))
        noc+=1
#     print("outgoing edge lisst :")

    incomingEdgeList = []
    for i in range(0, len(edgesList)):
        incomingEdgeList.append([])  # adding lists in a list
    for i in edges:
        v = edges[i][1]  # u -> v
        edge_no = int(i[2:-1])
        incomingEdgeList[v].append(edge_no)

#     print("incoming edge list: ", incomingEdgeList)
#     print()

    #  constraint : incoming for a customer
    for i in range(0, len(nodeCount)):  # choose one list from nodeCount
        smallest_node = nodeCount[i][0][0]
        largest_node = nodeCount[i][-1][-1]
        coeff = []
        for j in range(0, len(nodeCount[i])):
            for k in range(0,len(nodeCount[i][j])):
                node = nodeCount[i][j][k]
                for l in range(0,len(incomingEdgeList[node])):
                    e = incomingEdgeList[node][l] # edge number of incoming edge
                    
                    u = edges['x[%i]' % e][0]
                    v = edges['x[%i]' % e][1]
            
#                     print(node == v)
                    if(u<smallest_node or u>largest_node):
                        coeff.append(x[e])
        m1.addConstr(( sum(coeff) == 1  ))
        noc+=1

    # constraint for a node incoming = outgoing
    for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])):
            for k in range(0,len(nodeCount[i][j])):
                coeff = []
                node = nodeCount[i][j][k]
                for l in range(0, len(incomingEdgeList[node])):
                    e = incomingEdgeList[node][l]
                    coeff.append(x[e])
                for l in range(0, len(edgesList[node])):
                    e = edgesList[node][l]
                    coeff.append(-x[e])
                m1.addConstr(( sum(coeff) == 0 ))
                noc+=1  
            

               
    # #  0<= xij - fij    f<x => 0<=x-f
#     for j in range(len(flows)):
#         diff = offset*j+offset
#         for k in flows[j].keys():
#             coeff = [] 
#             coeff.append(-x[k])
#             coeff.append(x[k-diff])
#             m1.addConstr(( sum(coeff) >= 0 ))
            

    # constraint for a particular customer outgoing == 0 for c the customer for rest dont care
#     for c in range(customers):
#         addn = offset*(c+1)
#         for i in range(0, len(nodeCount)):
#             smallest_node = nodeCount[i][0][0]
#             largest_node = nodeCount[i][-1][-1]
#             if(i == c):    
#                 coeff = []
#                 for j in range(0, len(nodeCount[i])):
#                     for k in range(0,len(nodeCount[i][j])):
#                         node = nodeCount[i][j][k]
#                         for l in range(0,len(edgesList[node])):
#                             e = edgesList[node][l]
#                             u = edges['x[%i]' % e][0]
#                             v = edges['x[%i]' % e][1]
#                         if(v<smallest_node or v>largest_node):
#                             coeff.append(x[e+addn])
#                 m1.addConstr(( sum(coeff) == 0 ))


# #     print('done w edges')        






#     print("CONSTRAINT for incoming = 1 ")
#     for c in range(customers):
#             addn = offset*(c+1)
#             i = c
#             smallest_node = nodeCount[i][0][0]
#             largest_node = nodeCount[i][-1][-1]      
#             coeff = []
#             for j in range(0, len(nodeCount[i])):
#                     for k in range(0,len(nodeCount[i][j])):
#                         node = nodeCount[i][j][k]
#                         for l in range(0,len(incomingEdgeList[node])):
#                             e = incomingEdgeList[node][l]
#                             u = edges['x[%i]' % e][0]
#                             v = edges['x[%i]' % e][1]
#                         if(u<smallest_node or u>largest_node):
#                             coeff.append(x[e+addn])
#                             m1.addConstr(( sum(coeff) == 1 ))



#     # # flow conservation  for every node except those present in customer = c

#     for c in range(customers):
#         addn = offset*c+offset
#         for i in range(0, len(nodeCount)):
#             if ( i != c):
#                   for j in range(0, len(nodeCount[i])):
#                     for k in range(0,len(nodeCount[i][j])):
#                         coeff = []
#                         node = nodeCount[i][j][k]

#                         for l in range(0, len(incomingEdgeList[node])):
#                             e = incomingEdgeList[node][l]
#                             coeff.append(x[e+addn])
#                         for l in range(0, len(edgesList[node])):
#                             e = edgesList[node][l]
#                             coeff.append(-x[e+addn])
                            
#                         m1.addConstr(( sum(coeff) == 0 ))
#                         noc+=1  
                                                       
    
                                 
                                 
#     m1.params.Method = 1
#     m1.params.Crossover = 0     

    #minimizing the number of truck leaving S
    objectivefun = []  
    for i in range(0, len(edgesList[0])):
        objectivefun.append(x[edgesList[0][i]])
    obj = sum(objectivefun)
    m1.setObjective(obj,GRB.MINIMIZE)
 

    m1.optimize()
    total_runtime += m1.Runtime
    print("Runtime is :",m1.Runtime)
    print("Objective value : ",m1.objVal)
     
    # ans stores {1:0.5,4:0.9,...}   
    ans = {}
    for v in m1.getVars()[:offset]:
        if (v.x > 0):
            ans[(v.varName[2:-1])]=v.x

                                 
    new_edges = {}   # stores only those edges which are required   {e:(u,v)}
    for key in edges:
        if((key[2:-1]) in ans):
            new_edges[(key[2:-1])] = edges[key]
    
#     print(ans)
                                 
                                 
    graphForpaths = [] # will store outgoing edges
    for i in range(len(edgesList)): # edgesList contains outgoing from each node
        l = []
        for key in new_edges:
            if(new_edges[key][0] == i):  # u = i
                if(new_edges[key][1] == -1):
                    l.append(len(edgesList)-1)
                else:
                    l.append(new_edges[key][1])
        graphForpaths.append(l)
        
    allcustomer = [] # will store customers from longest path + ones path for del 
    # --------------------------------------------------- 
    # ONES PATH
    edges1=[] # stores outgoing edges wtih val > 0
    for i in range(nodeCount[-1][-1][-1]+1):
        edges1.append([])
    for key in new_edges.keys():
        edges1[new_edges[key][0]].append(key)
        
    paths = onesPath(graphForpaths,edges1)  # paths contains all ones path
    for i in range(len(paths)):
        paths[i][-1] = -1
#     print("Ones paths",paths)
   
    for route in paths:
        distict_customers,customer_nodes = distinctCustomer(route,nodeCount)
        customer_occurence = customer_nodes.copy()
        allcustomer+=customer_nodes
        customers_covered += distict_customers
        #----------------------------
        final_str = str()
        final_str = final_str + 'Ones path : Source -> '
    
       
        
        for i in customer_occurence:
            customer_number = searchKey(i,cust_dict)  # search key with value = i (i represents customer)
            final_str = final_str + ''.join(str(customer_number)+ ' -> ')


        final_str = final_str+' Sink '

        final_path_list.append(final_str)

        customer_nodes.sort()  # will store customers in ascending order
        for i in customer_nodes:
            for key, value in cust_dict.items():
                if i == value:
                    cust_dict[key]= -(VEHICLES+1)
        
        VEHICLES+=1
        #---------------------------------------------------------    
    #----------------------------------------------------
    
    
#     print("Edges",new_edges)
#     print("ANS",ans)
    visited = [False]*len(graphForpaths)
    ts = topologicalSort(visited,graphForpaths)
    
    while(True):
        # dp[i] stores maximum length from s to i
        path = [0]*len(ts)                     # stores index of path for which max occurs at i
        dp = [0]*len(ts)
        dp[0] = 1
        for i in range(1,len(ts)):
            u = ts[i]
            for j in range(0,i):
                v = ts[j]
                if(u in graphForpaths[v]): # egde to a prev node always exists
                    # v -> u and if path from s to v contains a node x st cust(x) = cust(u) then also add 0
                    val = length(u,v,nodeCount,path,j,ts)
                    if(val + dp[j] > dp[i]):
                        dp[i] = val + dp[j]
                        path[i] = j
    #             dp[i] = max(dp[i],length(u,v,nodeCount) + dp[j])
    #     print("max length :",dp[-1]-2)
    #     print(path)
    
#         print("index path:",path)
        route = []
        route.append(-1)
        curr = -1
        while(True):
            if(curr == 0):
                break
            prev = path[curr]
            route.append(ts[prev])
            curr = prev

        for l in range(1,len(route)-1):
            del ts[ts.index(route[l])]
            
            
        route.reverse()
        print("other routes :",route)
        if route not in paths:
            print("longest route (repeat):",route)
            break

    
    # now i need longest 
    distict_customers,customer_nodes = distinctCustomer(route,nodeCount)
    customer_occurence = customer_nodes.copy() # customer node stores index of visited customer in order of their occ in routes
    customers_covered+=distict_customers 
#     print("customer_occurence  -->",customer_occurence)
    
#     if(len(customer_nodes) == 0 and customers_covered == customers): # i.e no path other than one exist
#         print("breaking.......")
#         break
    if(len(customer_nodes) == 0): # i.e no path other than one exist
        print("Total customers covered",customers_covered)
        break
    
    
    
    # new addition-----------------
    final_str = str()
    final_str = final_str + 'Source -> '
    
    
#     print("customer dixt->",cust_dict)
    for i in customer_occurence:
        customer_number = searchKey(i,cust_dict)     # search key with value = i (i represents customer)
        final_str = final_str + ''.join(str(customer_number)+ ' -> ')
        
        
    final_str = final_str+' Sink '

    final_path_list.append(final_str)
    
    customer_nodes.sort()  # will store customers in ascending order
    for i in customer_nodes:
        for key, value in cust_dict.items():
            if i == value:
                cust_dict[key]= -(VEHICLES+1)
                
    a = 0
    for i in range(len(cust_dict)):
        if cust_dict[i]>= 0 :
            cust_dict[i] = a
            a+=1
    #----------------------------
    
    
    allcustomer += customer_nodes
    allcustomer = np.unique(allcustomer)
    allcustomer = allcustomer[::-1]

    #-------------------------
    #modify time-windows
    for i in range(0,len(allcustomer)):
        del timeWindows[allcustomer[i]]
        
    for i in range(0,len(allcustomer)):
        del time_from_source[allcustomer[i]]
        
    for i in range(0,len(allcustomer)):
        del demand[allcustomer[i]]
    #----------------------------------
    
                                 
    discreteTimeWindows = []
    for i in timeWindows:
        discreteTimeWindows.append(discreteTime(i, 10))
        
    capacities = [] #3x3 to store capacities , capacities[i][j][k] -> capacity of ith customer @ discreteTimeWindows[i][j]
    for i in range(0,len(discreteTimeWindows)):
        customer_i = []
        dc = discreteCapacities(solomon_capacity,10)
        for j in range(0,len(dc)):
            customer_i.append([dc[j]] * len(discreteTimeWindows[i]))
        capacities.append(customer_i)                        

    # removing visited customer from time matrix
    for row in allcustomer:
        del timeMatrix[row]

    for row in range(0,len(timeMatrix)):                         
        for col in allcustomer:
            del timeMatrix[row][col]
    
    print("                                                                                      ")
    print("Removing ",distict_customers," distinct customer",customer_nodes,"and path is ",route)
    print("----------------------------------------------------------------------------------------------------")
    print("                                                                                      ")
    
#     if(distict_customers != 0):
#         VEHICLES+=1 
    VEHICLES+=1 
    distance+=distance_cal(route,df)

In [ ]:
print("Vehicles required : ",VEHICLES)
print("Total Runtime",total_runtime)
print("Number of Iterations",Iterations)

In [ ]:
print(file)
final_path_list

In [ ]:
# R102.csv
# ['Source -> 91 -> 36 -> 1 -> 72 -> 21 -> 55 -> 73 -> 74 ->  Sink ',
#  'Source -> 95 -> 13 -> 43 -> 86 -> 56 -> 42 -> 90 -> 92 ->  Sink ',
#  'Source -> 51 -> 26 -> 68 -> 50 -> 80 -> 2 -> 67 -> 23 -> 79 ->  Sink ',
#  'Source -> 20 -> 38 -> 22 -> 66 -> 54 -> 3 -> 71 -> 57 ->  Sink ',
#  'Source -> 94 -> 97 -> 4 -> 98 -> 5 -> 25 -> 58 ->  Sink ',
#  'Ones path : Source -> 30 -> 62 -> 10 -> 18 -> 48 ->  Sink ',
#  'Source -> 82 -> 44 -> 60 -> 85 -> 93 -> 84 -> 99 ->  Sink ',
#  'Source -> 61 -> 87 -> 7 -> 45 -> 59 -> 88 ->  Sink ',
#  'Source -> 11 -> 27 -> 39 -> 52 -> 96 -> 12 -> 0 ->  Sink ',
#  'Source -> 49 -> 32 -> 29 -> 89 -> 9 -> 31 -> 81 ->  Sink ',
#  'Ones path : Source -> 64 -> 70 -> 8 -> 19 -> 76 ->  Sink ',
#  'Ones path : Source -> 63 -> 65 ->  Sink ',
#  'Ones path : Source -> 47 -> 37 -> 24 ->  Sink ',
#  'Ones path : Source -> 41 -> 14 -> 40 ->  Sink ',
#  'Ones path : Source -> 15 -> 83 -> 16 ->  Sink ',
#  'Source -> 6 -> 75 -> 78 -> 53 -> 33 ->  Sink ',
#  'Ones path : Source -> 35 -> 46 -> 17 -> 34 -> 69 ->  Sink ',
#  'Ones path : Source -> 28 -> 77 ->  Sink ']

# R103.csv
# ['Source -> 96 -> 41 -> 42 -> 14 -> 21 -> 55 -> 73 -> 72 -> 1 -> 57 ->  Sink ',
#  'Source -> 91 -> 36 -> 94 -> 39 -> 52 -> 67 -> 28 -> 79 -> 11 ->  Sink ',
#  'Source -> 90 -> 13 -> 43 -> 37 -> 15 -> 84 -> 95 -> 99 -> 58 ->  Sink ',
#  'Source -> 88 -> 51 -> 26 -> 75 -> 78 -> 9 -> 89 -> 69 -> 0 ->  Sink ',
#  'Source -> 17 -> 6 -> 61 -> 87 -> 83 -> 60 -> 16 -> 4 -> 92 ->  Sink ',
#  'Source -> 49 -> 32 -> 50 -> 29 -> 8 -> 70 -> 34 -> 76 -> 27 ->  Sink ',
#  'Source -> 82 -> 44 -> 98 -> 5 -> 59 -> 12 -> 93 -> 25 ->  Sink ',
#  'Source -> 23 -> 53 -> 74 -> 40 -> 54 -> 3 -> 24 -> 20 ->  Sink ',
#  'Source -> 31 -> 10 -> 7 -> 47 -> 18 -> 62 -> 30 ->  Sink ',
#  'Ones path : Source -> 35 -> 63 -> 48 -> 46 -> 45 -> 81 ->  Sink ',
#  'Source -> 80 -> 68 -> 86 -> 56 -> 97 -> 85 ->  Sink ',
#  'Source -> 38 -> 22 -> 66 -> 71 -> 2 ->  Sink ',
#  'Source -> 64 -> 19 -> 65 -> 33 ->  Sink ',
#  'Ones path : Source -> 77 ->  Sink ']

# R104.csv
# ['Source -> 27 -> 11 -> 67 -> 2 -> 75 -> 78 -> 80 -> 50 -> 29 -> 9 -> 30 ->  Sink ',
#  'Source -> 88 -> 95 -> 97 -> 90 -> 98 -> 83 -> 44 -> 82 -> 17 -> 51 -> 26 ->  Sink ',
#  'Source -> 96 -> 92 -> 94 -> 93 -> 52 -> 25 -> 39 -> 12 -> 5 -> 59 ->  Sink ',
#  'Source -> 91 -> 36 -> 41 -> 14 -> 42 -> 40 -> 73 -> 72 -> 1 -> 57 ->  Sink ',
#  'Source -> 87 -> 6 -> 61 -> 10 -> 63 -> 48 -> 35 -> 46 -> 47 ->  Sink ',
#  'Source -> 0 -> 69 -> 68 -> 32 -> 8 -> 34 -> 70 -> 65 -> 19 ->  Sink ',
#  'Source -> 58 -> 60 -> 16 -> 81 -> 7 -> 18 -> 89 -> 62 ->  Sink ',
#  'Source -> 21 -> 74 -> 22 -> 66 -> 38 -> 55 -> 3 -> 24 -> 20 ->  Sink ',
#  'Source -> 15 -> 85 -> 43 -> 37 -> 13 -> 4 -> 84 -> 99 ->  Sink ',
#  'Source -> 53 -> 28 -> 23 -> 79 -> 54 -> 76 -> 49 ->  Sink ',
#  'Source -> 64 -> 33 -> 77 -> 31 ->  Sink ',
#  'Ones path : Source -> 71 -> 56 -> 86 -> 45 ->  Sink ']

# R105.csv
# ['Source -> 91 -> 94 -> 97 -> 60 -> 15 -> 84 -> 96 -> 36 -> 90 -> 92 ->  Sink ',
#  'Ones path : Source -> 64 -> 70 -> 8 -> 77 -> 33 -> 34 ->  Sink ',
#  'Source -> 38 -> 22 -> 74 -> 66 -> 54 -> 53 -> 23 -> 79 ->  Sink ',
#  'Source -> 71 -> 1 -> 72 -> 21 -> 40 -> 55 -> 3 -> 24 ->  Sink ',
#  'Source -> 32 -> 11 -> 75 -> 78 -> 80 -> 2 -> 67 -> 57 ->  Sink ',
#  'Source -> 58 -> 51 -> 87 -> 6 -> 17 -> 45 -> 16 -> 99 ->  Sink ',
#  'Source -> 26 -> 27 -> 20 -> 39 -> 52 -> 49 -> 0 -> 69 ->  Sink ',
#  'Source -> 41 -> 13 -> 14 -> 56 -> 86 -> 95 -> 88 -> 59 ->  Sink ',
#  'Ones path : Source -> 4 -> 43 -> 37 -> 85 -> 42 ->  Sink ',
#  'Source -> 44 -> 82 -> 98 -> 93 -> 5 -> 25 -> 12 ->  Sink ',
#  'Source -> 30 -> 68 -> 29 -> 50 -> 19 -> 65 -> 76 ->  Sink ',
#  'Ones path : Source -> 62 -> 10 -> 61 -> 89 -> 9 -> 47 ->  Sink ',
#  'Ones path : Source -> 46 -> 18 -> 63 -> 48 -> 31 ->  Sink ',
#  'Ones path : Source -> 35 -> 81 -> 7 -> 73 ->  Sink ',
#  'Ones path : Source -> 28 -> 83 ->  Sink ']

# R106.csv
# ['Source -> 91 -> 1 -> 72 -> 21 -> 40 -> 74 -> 55 -> 73 -> 71 -> 20 ->  Sink ',
#  'Source -> 94 -> 36 -> 41 -> 14 -> 56 -> 86 -> 90 -> 99 -> 92 ->  Sink ',
#  'Source -> 44 -> 84 -> 60 -> 98 -> 5 -> 93 -> 12 -> 57 -> 25 ->  Sink ',
#  'Ones path : Source -> 38 -> 22 -> 66 -> 53 -> 54 -> 3 -> 24 ->  Sink ',
#  'Source -> 30 -> 68 -> 29 -> 50 -> 78 -> 67 -> 23 -> 79 -> 76 ->  Sink ',
#  'Source -> 13 -> 43 -> 15 -> 85 -> 83 -> 82 -> 59 -> 88 -> 51 ->  Sink ',
#  'Source -> 62 -> 10 -> 61 -> 18 -> 48 -> 47 -> 6 -> 81 ->  Sink ',
#  'Source -> 32 -> 28 -> 39 -> 52 -> 96 -> 97 -> 95 -> 58 ->  Sink ',
#  'Source -> 26 -> 27 -> 75 -> 77 -> 80 -> 65 -> 2 -> 49 ->  Sink ',
#  'Source -> 64 -> 70 -> 8 -> 19 -> 34 -> 33 ->  Sink ',
#  'Source -> 11 -> 0 -> 87 -> 17 -> 45 -> 16 -> 4 ->  Sink ',
#  'Source -> 46 -> 63 -> 89 -> 31 -> 69 ->  Sink ',
#  'Ones path : Source -> 37 -> 42 ->  Sink ',
#  'Ones path : Source -> 35 -> 7 -> 9 ->  Sink ']

# R107.csv
# ['Source -> 96 -> 41 -> 42 -> 14 -> 56 -> 86 -> 97 -> 15 -> 99 -> 95 -> 88 ->  Sink ',
#  'Source -> 94 -> 84 -> 43 -> 90 -> 98 -> 5 -> 12 -> 57 -> 25 -> 27 ->  Sink ',
#  'Source -> 51 -> 26 -> 75 -> 78 -> 77 -> 2 -> 28 -> 67 -> 76 -> 49 ->  Sink ',
#  'Source -> 59 -> 82 -> 44 -> 7 -> 83 -> 16 -> 4 -> 92 -> 93 ->  Sink ',
#  'Source -> 45 -> 46 -> 35 -> 48 -> 18 -> 47 -> 81 -> 6 -> 17 ->  Sink ',
#  'Source -> 20 -> 74 -> 22 -> 40 -> 21 -> 55 -> 3 -> 53 -> 79 -> 11 ->  Sink ',
#  'Source -> 32 -> 50 -> 29 -> 87 -> 9 -> 62 -> 31 -> 69 -> 0 ->  Sink ',
#  'Source -> 30 -> 61 -> 10 -> 63 -> 65 -> 19 -> 80 ->  Sink ',
#  'Source -> 91 -> 68 -> 52 -> 39 -> 72 -> 73 -> 71 -> 1 ->  Sink ',
#  'Ones path : Source -> 23 -> 38 -> 66 -> 54 -> 24 ->  Sink ',
#  'Source -> 58 -> 60 -> 85 -> 37 -> 13 -> 89 ->  Sink ',
#  'Source -> 70 -> 64 -> 8 -> 34 ->  Sink ',
#  'Source -> 36 -> 33 ->  Sink ']

# R108.csv
# ['Source -> 91 -> 90 -> 43 -> 85 -> 60 -> 83 -> 4 -> 82 -> 17 -> 51 -> 26 ->  Sink ',
#  'Source -> 96 -> 41 -> 42 -> 14 -> 56 -> 86 -> 94 -> 12 -> 57 -> 25 -> 27 ->  Sink ',
#  'Source -> 30 -> 68 -> 69 -> 29 -> 50 -> 8 -> 80 -> 32 -> 76 -> 79 ->  Sink ',
#  'Source -> 88 -> 95 -> 84 -> 36 -> 98 -> 7 -> 45 -> 46 -> 18 -> 47 ->  Sink ',
#  'Source -> 59 -> 5 -> 93 -> 52 -> 39 -> 72 -> 73 -> 40 -> 21 -> 20 ->  Sink ',
#  'Source -> 71 -> 74 -> 22 -> 38 -> 66 -> 54 -> 24 -> 53 -> 11 ->  Sink ',
#  'Source -> 49 -> 2 -> 67 -> 75 -> 78 -> 28 -> 23 -> 3 -> 55 ->  Sink ',
#  'Source -> 87 -> 9 -> 61 -> 10 -> 63 -> 89 -> 31 ->  Sink ',
#  'Source -> 58 -> 92 -> 97 -> 15 -> 37 -> 13 -> 99 ->  Sink ',
#  'Source -> 0 -> 70 -> 64 -> 34 -> 65 -> 19 ->  Sink ',
#  'Source -> 62 -> 81 -> 6 -> 1 ->  Sink ',
#  'Source -> 48 -> 35 ->  Sink ',
#  'Source -> 77 -> 33 ->  Sink ',
#  'Source -> 16 -> 44 ->  Sink ']

# R109.csv
# ['Source -> 94 -> 97 -> 15 -> 85 -> 60 -> 98 -> 5 -> 96 -> 99 -> 92 ->  Sink ',
#  'Source -> 58 -> 4 -> 82 -> 7 -> 44 -> 83 -> 16 -> 36 -> 90 -> 88 ->  Sink ',
#  'Source -> 27 -> 11 -> 75 -> 78 -> 28 -> 2 -> 49 -> 0 -> 69 ->  Sink ',
#  'Source -> 26 -> 51 -> 87 -> 61 -> 18 -> 46 -> 45 -> 95 -> 12 -> 57 ->  Sink ',
#  'Source -> 71 -> 74 -> 22 -> 38 -> 66 -> 54 -> 53 -> 23 -> 79 ->  Sink ',
#  'Source -> 91 -> 1 -> 72 -> 20 -> 21 -> 55 -> 3 -> 24 ->  Sink ',
#  'Source -> 68 -> 30 -> 6 -> 17 -> 52 -> 25 -> 67 -> 76 ->  Sink ',
#  'Source -> 41 -> 13 -> 37 -> 43 -> 84 -> 59 -> 47 ->  Sink ',
#  'Source -> 81 -> 35 -> 10 -> 62 -> 89 -> 9 -> 31 ->  Sink ',
#  'Source -> 29 -> 50 -> 80 -> 8 -> 77 -> 33 -> 34 ->  Sink ',
#  'Source -> 32 -> 39 -> 93 -> 86 -> 56 -> 42 ->  Sink ',
#  'Ones path : Source -> 14 -> 40 -> 73 ->  Sink ',
#  'Source -> 64 -> 70 -> 65 -> 19 ->  Sink ',
#  'Ones path : Source -> 63 -> 48 ->  Sink ']

# R110.csv
# ['Source -> 94 -> 97 -> 4 -> 5 -> 93 -> 86 -> 56 -> 14 -> 91 -> 12 ->  Sink ',
#  'Source -> 68 -> 30 -> 61 -> 18 -> 46 -> 45 -> 7 -> 44 -> 16 -> 59 ->  Sink ',
#  'Source -> 27 -> 11 -> 75 -> 78 -> 28 -> 23 -> 53 -> 54 -> 24 -> 3 ->  Sink ',
#  'Source -> 58 -> 98 -> 60 -> 15 -> 84 -> 95 -> 96 -> 36 -> 90 -> 92 ->  Sink ',
#  'Source -> 69 -> 29 -> 50 -> 80 -> 2 -> 77 -> 33 -> 67 -> 79 ->  Sink ',
#  'Source -> 1 -> 40 -> 21 -> 72 -> 39 -> 52 -> 49 -> 76 -> 0 ->  Sink ',
#  'Source -> 71 -> 20 -> 74 -> 22 -> 38 -> 66 -> 55 -> 73 ->  Sink ',
#  'Source -> 26 -> 51 -> 87 -> 6 -> 9 -> 89 -> 63 -> 47 ->  Sink ',
#  'Source -> 81 -> 82 -> 83 -> 85 -> 43 -> 42 -> 41 -> 99 ->  Sink ',
#  'Source -> 32 -> 34 -> 70 -> 8 -> 64 ->  Sink ',
#  'Source -> 13 -> 37 -> 25 -> 57 -> 88 ->  Sink ',
#  'Source -> 62 -> 10 -> 48 -> 35 -> 31 ->  Sink ',
#  'Source -> 17 -> 65 -> 19 ->  Sink ']

# R111.csv
# ['Source -> 94 -> 97 -> 36 -> 98 -> 60 -> 83 -> 16 -> 4 -> 92 -> 95 -> 88 ->  Sink ',
#  'Source -> 41 -> 14 -> 56 -> 86 -> 93 -> 5 -> 96 -> 12 -> 57 -> 1 ->  Sink ',
#  'Source -> 26 -> 51 -> 87 -> 61 -> 10 -> 62 -> 9 -> 89 -> 31 -> 69 ->  Sink ',
#  'Source -> 91 -> 90 -> 43 -> 85 -> 37 -> 42 -> 13 -> 99 -> 58 ->  Sink ',
#  'Source -> 20 -> 74 -> 22 -> 38 -> 66 -> 54 -> 53 -> 3 -> 24 ->  Sink ',
#  'Source -> 27 -> 32 -> 80 -> 77 -> 8 -> 34 -> 70 -> 65 -> 19 -> 0 ->  Sink ',
#  'Source -> 15 -> 84 -> 44 -> 7 -> 82 -> 17 -> 6 -> 47 -> 81 ->  Sink ',
#  'Ones path : Source -> 18 -> 63 -> 48 -> 35 -> 46 -> 45 -> 59 ->  Sink ',
#  'Source -> 30 -> 68 -> 75 -> 52 -> 39 -> 72 -> 73 -> 71 -> 25 ->  Sink ',
#  'Source -> 64 -> 78 -> 28 -> 23 -> 79 -> 76 -> 2 -> 49 ->  Sink ',
#  'Source -> 11 -> 40 -> 21 -> 55 -> 67 -> 33 ->  Sink ',
#  'Source -> 50 -> 29 ->  Sink ']

# R112.csv
# ['Source -> 94 -> 97 -> 15 -> 85 -> 43 -> 84 -> 4 -> 5 -> 95 -> 59 -> 88 ->  Sink ',
#  'Source -> 26 -> 51 -> 87 -> 6 -> 81 -> 17 -> 82 -> 83 -> 92 -> 96 -> 12 ->  Sink ',
#  'Source -> 91 -> 58 -> 93 -> 86 -> 56 -> 14 -> 42 -> 41 -> 13 -> 90 ->  Sink ',
#  'Source -> 68 -> 29 -> 50 -> 19 -> 65 -> 70 -> 8 -> 32 -> 49 -> 0 ->  Sink ',
#  'Source -> 1 -> 72 -> 71 -> 3 -> 53 -> 23 -> 28 -> 79 -> 76 -> 67 ->  Sink ',
#  'Source -> 30 -> 61 -> 18 -> 10 -> 63 -> 48 -> 35 -> 46 -> 47 ->  Sink ',
#  'Source -> 27 -> 11 -> 75 -> 69 -> 9 -> 62 -> 31 -> 89 ->  Sink ',
#  'Source -> 78 -> 80 -> 77 -> 33 -> 2 -> 25 -> 57 ->  Sink ',
#  'Source -> 20 -> 74 -> 22 -> 38 -> 66 -> 55 -> 73 ->  Sink ',
#  'Source -> 60 -> 98 -> 36 -> 16 -> 44 ->  Sink ',
#  'Source -> 39 -> 52 -> 37 -> 99 ->  Sink ',
#  'Source -> 40 -> 21 -> 54 -> 24 ->  Sink ',
#  'Source -> 64 -> 34 ->  Sink ',
#  'Source -> 7 -> 45 ->  Sink ']


In [ ]:
# # 50 
# R101.csv
# ['Ones path : Source -> 35 -> 10 -> 48 -> 31 ->  Sink ',
#  'Source -> 44 -> 46 -> 17 -> 9 -> 47 ->  Sink ',
#  'Ones path : Source -> 41 -> 43 -> 37 -> 42 ->  Sink ',
#  'Ones path : Source -> 38 -> 22 -> 21 -> 36 ->  Sink ',
#  'Ones path : Source -> 32 -> 28 -> 8 -> 49 -> 24 ->  Sink ',
#  'Ones path : Source -> 30 -> 18 -> 7 -> 45 -> 16 ->  Sink ',
#  'Ones path : Source -> 29 -> 33 -> 34 ->  Sink ',
#  'Ones path : Source -> 27 -> 6 -> 19 ->  Sink ',
#  'Ones path : Source -> 26 -> 11 -> 2 -> 12 ->  Sink ',
#  'Ones path : Source -> 13 -> 14 -> 40 -> 25 -> 0 ->  Sink ',
#  'Ones path : Source -> 4 -> 15 -> 5 -> 3 ->  Sink ',
#  'Ones path : Source -> 1 -> 20 -> 39 -> 23 ->  Sink ']

# R102.csv
# ['Source -> 49 -> 26 -> 6 -> 17 -> 9 -> 36 ->  Sink ',
#  'Source -> 11 -> 27 -> 25 -> 39 -> 20 -> 3 -> 24 ->  Sink ',
#  'Source -> 47 -> 46 -> 18 -> 7 -> 45 -> 16 ->  Sink ',
#  'Ones path : Source -> 44 -> 15 -> 5 -> 23 ->  Sink ',
#  'Ones path : Source -> 41 -> 43 -> 37 -> 33 ->  Sink ',
#  'Ones path : Source -> 38 -> 22 -> 40 -> 42 -> 30 ->  Sink ',
#  'Ones path : Source -> 35 -> 10 -> 48 -> 31 -> 0 ->  Sink ',
#  'Ones path : Source -> 32 -> 28 -> 19 -> 4 ->  Sink ',
#  'Ones path : Source -> 29 -> 8 -> 34 -> 2 ->  Sink ',
#  'Ones path : Source -> 13 -> 14 -> 21 -> 12 -> 1 ->  Sink ']

# R103.csv
# ['Source -> 27 -> 26 -> 25 -> 39 -> 11 -> 2 -> 33 -> 49 ->  Sink ',
#  'Source -> 17 -> 44 -> 45 -> 7 -> 9 -> 30 -> 31 ->  Sink ',
#  'Source -> 41 -> 42 -> 14 -> 21 -> 20 -> 3 -> 24 ->  Sink ',
#  'Source -> 47 -> 10 -> 18 -> 48 -> 46 -> 6 ->  Sink ',
#  'Source -> 36 -> 15 -> 43 -> 37 -> 13 -> 12 -> 4 ->  Sink ',
#  'Ones path : Source -> 0 -> 32 -> 29 -> 8 -> 34 -> 19 ->  Sink ',
#  'Source -> 38 -> 22 -> 40 -> 23 -> 28 ->  Sink ',
#  'Ones path : Source -> 35 -> 5 -> 16 -> 1 ->  Sink ']

# R104.csv
# ['Source -> 5 -> 4 -> 16 -> 44 -> 7 -> 45 -> 46 -> 18 -> 47 -> 6 ->  Sink ',
#  'Source -> 20 -> 21 -> 22 -> 38 -> 23 -> 28 -> 2 -> 32 -> 49 ->  Sink ',
#  'Source -> 41 -> 13 -> 43 -> 37 -> 15 -> 12 -> 1 -> 39 ->  Sink ',
#  'Source -> 30 -> 9 -> 10 -> 48 -> 29 -> 19 -> 0 ->  Sink ',
#  'Source -> 26 -> 36 -> 14 -> 42 -> 40 -> 3 -> 24 -> 27 ->  Sink ',
#  'Source -> 31 -> 34 -> 8 -> 33 -> 11 -> 25 ->  Sink ',
#  'Source -> 17 -> 35 ->  Sink ']

# R105.csv
# ['Source -> 41 -> 13 -> 43 -> 15 -> 5 -> 36 -> 16 ->  Sink ',
#  'Source -> 38 -> 22 -> 40 -> 21 -> 25 -> 3 -> 24 ->  Sink ',
#  'Source -> 44 -> 46 -> 35 -> 48 -> 9 -> 31 ->  Sink ',
#  'Ones path : Source -> 1 -> 14 -> 37 -> 42 -> 12 ->  Sink ',
#  'Source -> 27 -> 20 -> 39 -> 2 -> 49 -> 23 ->  Sink ',
#  'Source -> 30 -> 10 -> 18 -> 7 -> 45 ->  Sink ',
#  'Ones path : Source -> 32 -> 28 -> 11 -> 33 -> 34 ->  Sink ',
#  'Ones path : Source -> 26 -> 29 -> 8 -> 47 ->  Sink ',
#  'Ones path : Source -> 4 -> 17 -> 6 -> 19 -> 0 ->  Sink ']

# R106.csv
# ['Ones path : Source -> 13 -> 43 -> 15 -> 37 -> 42 -> 12 -> 1 ->  Sink ',
#  'Source -> 36 -> 41 -> 14 -> 40 -> 21 -> 20 -> 3 -> 24 ->  Sink ',
#  'Source -> 47 -> 46 -> 18 -> 10 -> 9 -> 19 -> 30 -> 0 ->  Sink ',
#  'Source -> 25 -> 27 -> 26 -> 5 -> 7 -> 45 -> 16 -> 4 ->  Sink ',
#  'Ones path : Source -> 49 -> 32 -> 2 -> 28 -> 8 -> 34 -> 11 ->  Sink ',
#  'Source -> 44 -> 35 -> 48 -> 31 -> 6 ->  Sink ',
#  'Source -> 38 -> 22 -> 39 -> 23 -> 33 ->  Sink ',
#  'Ones path : Source -> 29 -> 17 ->  Sink ']

# R107.csv
# ['Source -> 27 -> 26 -> 25 -> 39 -> 5 -> 4 -> 16 -> 15 -> 36 ->  Sink ',
#  'Source -> 41 -> 13 -> 43 -> 7 -> 45 -> 46 -> 47 -> 6 -> 17 ->  Sink ',
#  'Source -> 49 -> 32 -> 29 -> 19 -> 8 -> 34 -> 33 -> 2 ->  Sink ',
#  'Source -> 20 -> 38 -> 22 -> 40 -> 21 -> 3 -> 24 -> 11 ->  Sink ',
#  'Source -> 44 -> 18 -> 10 -> 9 -> 28 -> 23 ->  Sink ',
#  'Ones path : Source -> 42 -> 14 -> 37 -> 12 -> 1 ->  Sink ',
#  'Ones path : Source -> 35 -> 48 -> 31 -> 0 -> 30 ->  Sink ']

# R108.csv
# ['Source -> 5 -> 36 -> 41 -> 14 -> 42 -> 1 -> 12 -> 39 -> 25 -> 27 ->  Sink ',
#  'Source -> 0 -> 49 -> 2 -> 32 -> 8 -> 34 -> 19 -> 29 -> 9 -> 30 ->  Sink ',
#  'Source -> 44 -> 16 -> 7 -> 45 -> 46 -> 18 -> 47 -> 6 -> 17 ->  Sink ',
#  'Source -> 20 -> 21 -> 40 -> 22 -> 38 -> 3 -> 24 -> 11 ->  Sink ',
#  'Source -> 26 -> 31 -> 10 -> 48 -> 35 ->  Sink ',
#  'Source -> 4 -> 15 -> 43 -> 37 -> 13 ->  Sink ',
#  'Source -> 33 -> 28 -> 23 ->  Sink ']

# R109.csv
# ['Source -> 26 -> 27 -> 11 -> 28 -> 2 -> 49 -> 33 -> 34 ->  Sink ',
#  'Source -> 41 -> 13 -> 43 -> 15 -> 37 -> 36 -> 16 ->  Sink ',
#  'Source -> 30 -> 10 -> 18 -> 46 -> 35 -> 48 -> 31 ->  Sink ',
#  'Source -> 38 -> 22 -> 20 -> 21 -> 40 -> 42 -> 12 ->  Sink ',
#  'Source -> 32 -> 29 -> 8 -> 19 -> 9 -> 47 ->  Sink ',
#  'Source -> 4 -> 44 -> 6 -> 17 -> 5 -> 25 -> 0 ->  Sink ',
#  'Source -> 14 -> 1 -> 39 -> 23 -> 24 ->  Sink ',
#  'Source -> 7 -> 45 -> 3 ->  Sink ']

# R110.csv
# ['Source -> 1 -> 40 -> 21 -> 20 -> 39 -> 25 -> 49 -> 32 -> 0 ->  Sink ',
#  'Source -> 13 -> 43 -> 15 -> 37 -> 42 -> 14 -> 41 -> 12 ->  Sink ',
#  'Source -> 27 -> 30 -> 6 -> 17 -> 36 -> 5 -> 3 ->  Sink ',
#  'Ones path : Source -> 26 -> 11 -> 28 -> 33 -> 2 -> 23 -> 24 ->  Sink ',
#  'Source -> 46 -> 18 -> 10 -> 9 -> 29 -> 19 -> 31 ->  Sink ',
#  'Source -> 4 -> 44 -> 48 -> 35 -> 47 ->  Sink ',
#  'Source -> 16 -> 7 -> 45 ->  Sink ',
#  'Source -> 22 -> 38 ->  Sink ',
#  'Source -> 8 -> 34 ->  Sink ']

# R111.csv
# ['Source -> 26 -> 6 -> 17 -> 44 -> 7 -> 5 -> 12 -> 1 -> 25 ->  Sink ',
#  'Source -> 27 -> 11 -> 28 -> 2 -> 32 -> 8 -> 34 -> 33 -> 0 ->  Sink ',
#  'Source -> 47 -> 18 -> 10 -> 48 -> 35 -> 45 -> 16 -> 4 ->  Sink ',
#  'Source -> 46 -> 31 -> 29 -> 9 -> 30 -> 19 -> 49 ->  Sink ',
#  'Source -> 38 -> 22 -> 40 -> 39 -> 20 -> 3 -> 23 ->  Sink ',
#  'Ones path : Source -> 41 -> 14 -> 21 -> 24 ->  Sink ',
#  'Ones path : Source -> 13 -> 43 -> 15 -> 37 -> 42 -> 36 ->  Sink ']

# R112.csv
# ['Source -> 27 -> 26 -> 5 -> 4 -> 16 -> 44 -> 7 -> 45 -> 46 -> 47 ->  Sink ',
#  'Source -> 1 -> 39 -> 20 -> 25 -> 11 -> 2 -> 49 -> 0 ->  Sink ',
#  'Source -> 32 -> 8 -> 34 -> 33 -> 28 -> 23 -> 24 -> 3 ->  Sink ',
#  'Source -> 30 -> 17 -> 6 -> 9 -> 31 -> 19 ->  Sink ',
#  'Source -> 15 -> 43 -> 13 -> 41 -> 14 -> 42 -> 37 ->  Sink ',
#  'Source -> 29 -> 10 -> 18 -> 48 -> 35 ->  Sink ',
#  'Source -> 38 -> 40 -> 21 -> 36 -> 12 ->  Sink ',
#  'Ones path : Source -> 22 ->  Sink ']

In [ ]:
# R101.csv
# ['Ones path : Source -> 17 -> 9 ->  Sink ',
#  'Ones path : Source -> 10 -> 6 -> 3 ->  Sink ',
#  'Source -> 13 -> 14 -> 21 -> 16 ->  Sink ',
#  'Ones path : Source -> 22 -> 2 -> 0 ->  Sink ',
#  'Ones path : Source -> 15 -> 5 -> 23 ->  Sink ',
#  'Ones path : Source -> 11 -> 8 -> 19 ->  Sink ',
#  'Ones path : Source -> 4 -> 18 -> 7 -> 24 ->  Sink ',
#  'Ones path : Source -> 1 -> 20 -> 12 ->  Sink ']

# R102.csv
# ['Source -> 1 -> 20 -> 22 -> 21 -> 23 -> 24 ->  Sink ',
#  'Ones path : Source -> 17 -> 3 -> 2 ->  Sink ',
#  'Ones path : Source -> 15 -> 5 -> 16 ->  Sink ',
#  'Ones path : Source -> 8 -> 19 -> 4 ->  Sink ',
#  'Source -> 6 -> 10 -> 18 -> 9 -> 11 ->  Sink ',
#  'Ones path : Source -> 13 -> 14 -> 0 ->  Sink ',
#  'Ones path : Source -> 7 -> 12 ->  Sink ']

# R103.csv
# ['Ones path : Source -> 23 -> 22 -> 21 -> 3 -> 24 -> 20 ->  Sink ',
#  'Ones path : Source -> 17 -> 6 -> 10 -> 7 -> 4 -> 16 -> 15 ->  Sink ',
#  'Ones path : Source -> 13 -> 14 -> 5 -> 1 -> 12 -> 11 ->  Sink ',
#  'Ones path : Source -> 2 -> 19 -> 8 -> 9 -> 18 -> 0 ->  Sink ']

# R104.csv
# ['Source -> 5 -> 4 -> 14 -> 1 -> 3 -> 24 -> 20 ->  Sink ',
#  'Ones path : Source -> 21 -> 22 -> 13 -> 12 -> 0 ->  Sink ',
#  'Ones path : Source -> 18 -> 10 -> 7 -> 15 -> 16 -> 17 ->  Sink ',
#  'Ones path : Source -> 11 -> 23 -> 2 -> 8 -> 19 -> 9 -> 6 ->  Sink ']

# R105.csv
# ['Source -> 4 -> 13 -> 15 -> 5 -> 3 -> 24 ->  Sink ',
#  'Source -> 1 -> 17 -> 6 -> 9 -> 16 ->  Sink ',
#  'Ones path : Source -> 11 -> 8 -> 19 ->  Sink ',
#  'Ones path : Source -> 10 -> 18 -> 7 -> 0 ->  Sink ',
#  'Source -> 20 -> 22 -> 21 -> 12 ->  Sink ',
#  'Ones path : Source -> 14 -> 2 -> 23 ->  Sink ']

# R106.csv
# ['Source -> 13 -> 15 -> 17 -> 9 -> 0 -> 11 ->  Sink ',
#  'Source -> 2 -> 22 -> 21 -> 3 -> 12 -> 1 ->  Sink ',
#  'Source -> 20 -> 14 -> 5 -> 23 -> 24 ->  Sink ',
#  'Source -> 6 -> 10 -> 18 -> 7 -> 4 -> 16 ->  Sink ',
#  'Ones path : Source -> 8 -> 19 ->  Sink ']

# R107.csv
# ['Source -> 17 -> 6 -> 7 -> 5 -> 4 -> 16 -> 15 ->  Sink ',
#  'Ones path : Source -> 18 -> 10 -> 8 -> 9 -> 19 -> 0 ->  Sink ',
#  'Source -> 13 -> 14 -> 21 -> 20 -> 3 -> 24 -> 23 ->  Sink ',
#  'Ones path : Source -> 11 -> 22 -> 2 -> 12 -> 1 ->  Sink ']

# R108.csv
# ['Source -> 17 -> 6 -> 10 -> 18 -> 7 -> 5 -> 12 -> 11 ->  Sink ',
#  'Source -> 21 -> 20 -> 22 -> 3 -> 24 -> 23 ->  Sink ',
#  'Source -> 13 -> 14 -> 4 -> 16 -> 15 -> 1 ->  Sink ',
#  'Ones path : Source -> 9 -> 2 -> 8 -> 19 -> 0 ->  Sink ']

# R109.csv
# ['Source -> 1 -> 20 -> 21 -> 22 -> 24 -> 3 -> 23 ->  Sink ',
#  'Source -> 13 -> 15 -> 4 -> 7 -> 16 -> 12 ->  Sink ',
#  'Source -> 14 -> 17 -> 6 -> 9 -> 0 ->  Sink ',
#  'Ones path : Source -> 18 -> 10 -> 8 -> 19 ->  Sink ',
#  'Ones path : Source -> 11 -> 5 -> 2 ->  Sink ']

# R110.csv
# ['Source -> 1 -> 14 -> 21 -> 20 -> 2 -> 23 -> 3 ->  Sink ',
#  'Source -> 4 -> 6 -> 17 -> 7 -> 9 -> 0 ->  Sink ',
#  'Source -> 11 -> 5 -> 15 -> 13 -> 16 ->  Sink ',
#  'Ones path : Source -> 10 -> 18 -> 8 -> 19 -> 12 ->  Sink ',
#  'Source -> 22 -> 24 ->  Sink ']

# R111.csv
# ['Source -> 15 -> 13 -> 14 -> 21 -> 1 -> 12 -> 0 ->  Sink ',
#  'Source -> 20 -> 22 -> 2 -> 23 -> 3 -> 24 ->  Sink ',
#  'Source -> 6 -> 17 -> 5 -> 7 -> 4 -> 16 ->  Sink ',
#  'Source -> 18 -> 10 -> 9 -> 19 -> 11 ->  Sink ',
#  'Ones path : Source -> 8 ->  Sink ']

# R112.csv
# ['Source -> 15 -> 13 -> 14 -> 21 -> 22 -> 24 -> 3 ->  Sink ',
#  'Source -> 20 -> 11 -> 1 -> 12 -> 5 -> 4 -> 16 ->  Sink ',
#  'Source -> 6 -> 17 -> 7 -> 18 -> 9 -> 19 -> 0 ->  Sink ',
#  'Source -> 10 -> 8 -> 2 -> 23 ->  Sink ']

##### 